In [306]:
# import libraries
import pandas as pd
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# project specific libs
import os
import matplotlib.pyplot as plt

In [307]:
# set (local) path
path = '/Users/KevinBu/Desktop/clemente_lab/Projects/ampaim/'

In [308]:
###
# Load AB mapping file
###

# get AMPAIM+EISER mapping file w metadata; drop row 1
df_map_AB = pd.read_csv(path + 'inputs/adamcantor22_Cross_Disease_Pilot_0/Qiime2_0/qiime_mapping_file.tsv', sep='\t', header=0, index_col=0)

# get first row for any future operations
first_row = df_map_AB.reset_index().iloc[0,]
first_row_df = pd.DataFrame(first_row).T
df_map_AB = df_map_AB.iloc[1:,]


###
# Specific replacements
###

# 526-0-twin-psaplate308 is missing a dash
df_map_AB = df_map_AB.rename(index={'526-0-twin-psaplate308': '526-0-twin-psa-plate308'})

###
# General replacements
###

# refrain from dropping NA's because you might need the columns for Q2 and you need to smoothly concat the first row

# convert index to str from float etc.
df_map_AB.index = df_map_AB.index.map(str)

# switch EISER diagnosis to eiser from NA
df_map_AB.loc[df_map_AB['Project'] == 'eiser', 'Diagnosis'] = 'eiser'

# RBB mapping file doesn't have info on duplicate controls, and the glass control info is extracted only from a subset of the names
# so we need to grab that info from AB's mapping file
samp_to_ctrlstatus = df_map_AB['Project'].to_dict()

# grab AB non-microteach samples
df_map_AB_filt = df_map_AB[df_map_AB['Project'].isin(['eiser','TWIN_PSA','glass_control','neg_control','duplicate_control'])]

df_map_AB_filt.head()

,BarcodeSequence,LinkerPrimerSequence,AmpliconWell,BSA,BSASeverityByBSA,CCPtiter,CRP,CurrentBiologics,CurrentIntralesionalSteroids,CurrentMTX,...,StudyType,SubjectType,BirthYear,HostSubjectId,Nationality,Sex,SpecimenType,UberonCodeType,Weight,WeightDateCollected
#SampleID,,,,,,,,,,,,,,,,,,,,,
116783-plate305,CCTCGCATGACC,CCGGACTACHVGGGTWTCTAAT,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116783,NaN,NaN,fecal,NaN,NaN,NaN
116784-plate305,CGCGCAAGTATT,CCGGACTACHVGGGTWTCTAAT,B1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116784,NaN,NaN,fecal,NaN,NaN,NaN
116785-plate305,AAGGCGCTCCTT,CCGGACTACHVGGGTWTCTAAT,C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116785,NaN,NaN,fecal,NaN,NaN,NaN
116786-plate305,CGCAATGAGGGA,CCGGACTACHVGGGTWTCTAAT,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116786,NaN,NaN,fecal,NaN,NaN,NaN
116787-plate305,ACGGCGTTATGT,CCGGACTACHVGGGTWTCTAAT,E1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116787,NaN,NaN,fecal,NaN,NaN,NaN


In [318]:
###
# Prepare for merge with RB mapping file
###

# load RB mapping file
df_map_RB = pd.read_csv(path + 'inputs/qiime_mapping_file_final_062123_RBB_ESR_CRPmod.csv', index_col=0)

# skip row with q2types; 140 columns
df_map_RB = df_map_RB.iloc[1:,]

# exclude unaffected since we are using TWINS_PSA from AB and controls from AB
# we only want microteach samples
df_map_RB = df_map_RB[df_map_RB['Diagnosis'].isin(['healthy','ss','sle','cd','RA','psa','pso'])]

# drop duplicate samples that appear in AB and that are controls per AB mapping file
for s in list(df_map_RB.index.values):
    if samp_to_ctrlstatus[s] in ['glass_control','duplicate_control','neg_control']:
        df_map_RB = df_map_RB.drop(s)

        
# merge with AB
df_meta = pd.concat([df_map_AB_filt, df_map_RB], axis=0)

# drop 'Separate' and 'Together'; presumably Q2 artifacts
df_meta = df_meta.drop(['Separate','Together'], axis=1)

# add in Project col from AB that describes controls
df_meta['Project'] = df_meta.index.map(samp_to_ctrlstatus)

### 
# General replacements
###

# specify float cols for later
float_cols = ['Age','BSA','CRP','DAS28','ESR','PhysicianGlobalPsA','RAPID3','SJC','TJC']

# Replace Y,N with 1,0 
df_meta = df_meta.replace({'Y':1,'N':0})

# create dict mapping diagnosis to samples
diag_to_samp = {}
for d in list(set(df_meta['Diagnosis'].values)):
    diag_to_samp[d] = list(df_meta[df_meta['Diagnosis'] == d].index.values)

# create disease groupings
# (i) SS + SLE, PsO + PsA, CD, RA
# (ii) SS + SLE, PsO, PsA + RA, CD 

disease_to_dg1 ={
    'ss': 'SS_SLE',
    'sle': 'SS_SLE',
    'pso': 'PsA_PsO',
    'psa': 'PsA_PsO',
    'cd': 'CD',
    'RA': 'RA'
}

disease_to_dg2 ={
    'ss': 'SS_SLE',
    'sle': 'SS_SLE',
    'pso': 'PsO',
    'psa': 'PsA_RA',
    'cd': 'CD',
    'RA': 'PsA_RA'
}

df_meta['disease_grouping_1'] = df_meta['Diagnosis'].map(disease_to_dg1)
df_meta['disease_grouping_2'] = df_meta['Diagnosis'].map(disease_to_dg2)

# 361 rows
df_meta.head()

,BarcodeSequence,LinkerPrimerSequence,AmpliconWell,BSA,BSASeverityByBSA,CCPtiter,CRP,CurrentBiologics,CurrentIntralesionalSteroids,CurrentMTX,...,UberonCodeType,Weight,WeightDateCollected,CCPpos,Disease,DiseaseDurationYears,OtherSubjectID,RFpos,disease_grouping_1,disease_grouping_2
#SampleID,,,,,,,,,,,,,,,,,,,,,
116783-plate305,CCTCGCATGACC,CCGGACTACHVGGGTWTCTAAT,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116784-plate305,CGCGCAAGTATT,CCGGACTACHVGGGTWTCTAAT,B1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116785-plate305,AAGGCGCTCCTT,CCGGACTACHVGGGTWTCTAAT,C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116786-plate305,CGCAATGAGGGA,CCGGACTACHVGGGTWTCTAAT,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116787-plate305,ACGGCGTTATGT,CCGGACTACHVGGGTWTCTAAT,E1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [320]:
### 
# Create new mapping files
### 

# (1) complete mapping with controls
# concat the first row
df_map_filtered = pd.concat([first_row_df, df_meta])

# convert to CSV for Q2
df_map_filtered.to_csv(path + 'inputs/qiime_mapping_file_.tsv', sep='\t', index=False, na_rep='nan')

# (2) filter out controls into new metadat 
# should have 194 eiser + 106 microteach + 36 TWIN_PSA = 336 samples
df_meta = df_meta[df_meta['Project'].isin(['eiser','microteach','TWIN_PSA'])]

# concat the first row
df_map_filt = pd.concat([first_row_df, df_map_filt])

# convert to CSV for Q2
df_map_filt.to_csv(path + 'inputs/qiime_mapping_file_noctrl.tsv', sep='\t', index=False, na_rep='nan')

# drop eiser samples from downstream analysis
df_meta = df_meta[df_meta['Diagnosis'] != 'eiser']

# export to Q2
df_meta.to_csv(path + 'inputs/qiime_mapping_file_noctrl_noeiser.tsv', sep='\t', index=False, na_rep='nan') 

###
# Pre-processing
###


# drop eiser samples from downstream analysis
# df_meta = df_meta[df_meta['Diagnosis'] != 'eiser']

    

    
df_meta['Diagnosis'].value_counts()

RA            44
psa           36
pso           29
unaffected    14
healthy        6
ss             5
cd             4
sle            4
Name: Diagnosis, dtype: int64

In [6]:
### Loop for all LEfSe
# for LEfSe, jobs00, aggregate
# pairs, [<list of group 1> <list of group 2>]
### Loop for all LEfSe
# for LEfSe, jobs00, aggregate
# pairs, [<list of group 1> <list of group 2> <aggregate group 2>]
comp_to_groups = {
    'AvH': (['healthy'],['psa','pso','ss','cd','sle','RA'],['affected']), # healthy vs disease aggregate, jobs03
    'RAvH': (['healthy'],['RA'],['RA']), # jobs04 
    'PSvH': (['healthy'],['psa','pso'],['PS']), #jobs05
    'SSvH': (['healthy'],['ss'],['ss']), #jobs06
    'SLEvH': (['healthy'],['sle'],['sle']),
    'CDvH': (['healthy'],['cd'],['cd']), 
    'SSSLEvH': (['healthy'],['ss','sle'],['SS_SLE']),
    'RAPsAvH': (['healthy'],['RA','PsA'],['RA_PsA'])
    # SS + SLE, PsO + PsA, CD, RA
    # RA and PsA, SS + SLE, CD, PsO 
}

for c in comp_to_groups:
    # unpack tuples
    healthy, disease, aggregate = comp_to_groups[c]
    
    # subset relevant diseases
    df_meta_XvH = df_meta[df_meta['Diagnosis'].isin(healthy + disease)]
    
    # change all diseases to aggregate string
    df_meta_XvH.loc[df_meta_XvH['Diagnosis'].isin(disease), 'Diagnosis'] = aggregate

    # add first row
    df_meta_XvH = df_meta_XvH.reset_index()
    df_meta_XvH = pd.concat([first_row_df, df_meta_XvH])
    df_meta_XvH = df_meta_XvH.set_index('#SampleID')

    # export to csv
    df_meta_XvH.to_csv(path + 'inputs/qiime_mapping_file_noctrl_' + c + '.tsv', sep='\t', na_rep='nan')
    df_meta_XvH.head()



/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [9]:
# plot marginal dist
try:
    os.mkdir(path + 'outputs/marginal/')
except:
    pass

for x in df_meta.columns.values:
    # create a seaborn plot
    sns.set(style="darkgrid")
    ax = sns.displot(df_meta, x=x)

    # save the plot as PNG file
    plt.savefig(path + 'outputs/marginal/' + x + '.png')
    plt.close()

print('Done')

Done


In [16]:
# based on marginal analysis, variables that need to be standardized
# [‘BSA’, ‘CCPtiter’, ?‘DAS28’, ?‘DiseaseDuration’, ‘PhysicianGlobalPSA’, ?SJC, ?TJC]

# (1) BSA
# df_meta['BSA'].values
# how can 45214 and 10-May (7-Jan in RBB, would assume that's 1.7 with XL autocorrect) 
# be correct for '240-psa-plate307' and '241-pso-plate307' respectively?
# Specify <5 when values are 1, 2, 3

# (2) CCPtiter
# df_meta['CCPtiter'].values
# 25502.3 and 5156.2 seem outliers for '607-B-plate307' and 'EARA10-V1-plate307'
# Also specify <8 when 1, 0.5, exist - what to do? treat as 8?
# df_meta[df_meta['CCPtiter'].isin(['25502.3','5156.2'])]

# (3) DAS28, DiseaseDuration, PhysicianGlobalPsA, SJC, TJC look ok
df_meta['BSA']

#SampleID
131-slesjo-plate308        NaN
209-pso-plate307            <5
235-psa-plate307            <5
240-psa-plate307         45214
241-pso-plate307        10-May
                         ...  
prep032-pso-plate307       1.5
prep040-pso-plate307       NaN
prep047-pso-plate307      0.25
prep048-pso-plate307         5
prep053-pso-plate307         3
Name: BSA, Length: 142, dtype: object

In [7]:
df_map_RB = pd.read_csv(path + 'inputs/qiime_mapping_file_final_062123_RBB_ESR_CRPmod.csv', index_col=0)
df_map_RB = df_map_RB.iloc[1:,]

RB_samp = df_map_RB.index.values
KB_samp = df_meta.index.values
shared_samp = set(RB_samp).intersection(set(KB_samp))

# diff_samp yields many twin samples not included
diff_samp = set(RB_samp) ^ set(KB_samp)

print(len(RB_samp), len(KB_samp), len(shared_samp))
df_sub_RB = df_map_RB.loc[shared_samp]
df_sub_KB = df_meta.loc[shared_samp]

# comapre sub_dfs
# for every column
for c in list(df_sub_RB.columns.values):
    if c in df_sub_KB.columns.values and c in df_sub_RB.columns.values:
        if c not in ['ResultsProtocolID', 'SampleProtocolID', 'RawDataNotes', 'StudyName']:
            # for every sample
            for s in shared_samp:
                if df_sub_KB[c].loc[s] != df_sub_RB[c].loc[s] and not pd.isna(df_sub_KB[c].loc[s]):
                    try:
                        if float(df_sub_KB[c].loc[s]) != float(df_sub_RB[c].loc[s]):
                            print(c, s)
                            print(df_sub_KB[c].loc[s], df_sub_RB[c].loc[s])
                    except:
                        print(c, s)
                        print(df_sub_KB[c].loc[s], df_sub_RB[c].loc[s])
    else:
        print(c)

not pd.isna(df_sub_KB[c].loc[s])

128 142 117
CCPpos
Dactylitis 531-0-twin-psa-plate308
0 N
Dactylitis 541-0-twin-psa-plate308
0 N
Dactylitis prep047-pso-plate307
0 N
Dactylitis 537-0-twin-psa-plate308
0 N
Dactylitis 519-0-twin-psa-plate308
0 N
Dactylitis 533-0-twin-psa-plate308
0 N
Dactylitis 527-2-twin-psa-plate308
0 N
Dactylitis 539-0-twin-psa-plate308
0 N
Dactylitis 545-0-twin-psa-plate308
0 N
Dactylitis prep032-pso-plate307
0 N
Dactylitis prep053-pso-plate307
0 N
Dactylitis 543-0-twin-psa-plate308
0 N
Dactylitis 521-0-twin-psa-plate308
0 N
Dactylitis 535-0-twin-psa-plate308
0 N
Disease
DiseaseDurationYears
Enthesitis prep047-pso-plate307
0 N
Enthesitis prep053-pso-plate307
0 N
OtherSubjectID
RFpos
Race 607-B-plate307
black/afro-carribean black
Separate 241-pso-plate307
All Limit 45 Characters
Separate 209-pso-plate307
All Text
StoolReceivedDate 533-0-twin-psa-plate308
9/18/2019 9/18/19
AbxInPast3Months 483-B-plate307
UNCLEAR (taken in the past year) nan
AbxInPast3Months 607-B-plate307
UNCLEAR, IN PAST YEAR nan
Abx

False

In [128]:
# plot marginal dist of float vars only :)
try:
    os.mkdir(path + 'outputs/marginal_numeric/')
except:
    pass

for x in float_cols:
    # create a seaborn plot
    sns.set(style="darkgrid")
    ax = sns.displot(df_meta, x=x)

    # save the plot as PNG file
    plt.savefig(path + 'outputs/marginal_numeric/' + x + '.png')
    plt.close()

print('Done')

Done


In [154]:
# unnormalized OTU table level-6 also has metadata in it! nice 
# from taxa_bar_plot.qzv
df_otu = pd.read_csv(path + 'inputs/adamcantor22_Cross_Disease_Pilot_0//Qiime2_0/level-6.csv', index_col=0)

# determine columns to drop; i.e. keep taxa only
dropcol = []
for c in list(df_otu.columns.values):
    if c[0:3] != 'k__':
        dropcol.append(c)
        
df_otu = df_otu.drop(dropcol, axis=1)

# normalize the cols
df_otu = df_otu.div(df_otu.sum(axis=1), axis=0)
df_otu.head()

,k__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanobrevibacter,k__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanosphaera,k__Archaea;p__Euryarchaeota;c__Thermoplasmata;o__E2;f__[Methanomassiliicoccaceae];g__,k__Archaea;p__Euryarchaeota;c__Thermoplasmata;o__E2;f__[Methanomassiliicoccaceae];g__Methanomassiliicoccus,k__Archaea;p__Euryarchaeota;c__Thermoplasmata;o__E2;f__[Methanomassiliicoccaceae];g__vadinCA11,k__Bacteria;__;__;__;__;__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;__;__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Varibaculum,...,k__Bacteria;p__TM7;c__TM7-3;o__CW040;f__;g__,k__Bacteria;p__Tenericutes;__;__;__;__,k__Bacteria;p__Tenericutes;c__Mollicutes;o__Anaeroplasmatales;f__Anaeroplasmataceae;g__,k__Bacteria;p__Tenericutes;c__Mollicutes;o__RF39;f__;g__,k__Bacteria;p__Tenericutes;c__RF3;o__ML615J-28;f__;g__,k__Bacteria;p__Verrucomicrobia;c__Opitutae;__;__;__,k__Bacteria;p__Verrucomicrobia;c__Opitutae;o__[Cerasicoccales];f__[Cerasicoccaceae];g__,k__Bacteria;p__Verrucomicrobia;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Verrucomicrobiaceae;g__Akkermansia,k__Bacteria;p__Verrucomicrobia;c__[Pedosphaerae];o__[Pedosphaerales];f__R4-41B;g__,k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Deinococcaceae;g__Deinococcus
index,,,,,,,,,,,,,,,,,,,,,
116783-plate305,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000203,0.0,0.0
116784-plate305,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
116785-plate305,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
116786-plate305,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
116787-plate305,0.0,0.0,0.0,0.0,0.0,0.000523,0.0,0.0,0.0,0.0,...,0.0,0.000523,0.0,0.002442,0.0,0.0,0.001046,0.043146,0.0,0.0


In [236]:
# PICRUST2
# don't forget to uncompress path_abun_unstrat.tsv.gz
# 393 columns/pathways represented here
df_fxn = pd.read_csv(path + 'inputs/adamcantor22_Cross_Disease_Pilot_0/PiCRUSt2_0/picrust2_out/pathways_out/path_abun_unstrat.tsv', sep='\t', index_col=0).T
KO_names = list(df_fxn.columns.values)

# normalize by row total (per sample basis)
df_fxn = df_fxn.div(df_fxn.sum(axis=1), axis=0)
df_fxn.to_csv(path + 'inputs/df_pathways.tsv', sep='\t')

# new metadata for concatenation
# df_meta_sample = df_meta.set_index('SampleID')

# concatenate
# df_pi = pd.concat([df_meta_sample, df_fxn], axis=1)
# df_pi.head()

#gene names; KO is just a placeholder
# KO_names = [x for x in df_pi.columns.values if x[0] == 'K']
# KO_names = [x for x in df_pi.columns.values if x[-3:] == 'PWY']

# RA first
# df_KO = df_pi[df_pi['Diagnosis'].isin(['healthy','RA'])]#'pso','psa'])]
# df_KO = df_KO[KO_names + ['Diagnosis']]

# subset
# df_ra_KO = df_KO[df_KO['Diagnosis'].isin(['RA'])]#'pso','psa'])]
# df_healthy_KO = df_KO[df_KO['Diagnosis'].isin(['healthy'])]


df_fxn.head()

pathway,1CMET2-PWY,3-HYDROXYPHENYLACETATE-DEGRADATION-PWY,ALL-CHORISMATE-PWY,ANAEROFRUCAT-PWY,ANAGLYCOLYSIS-PWY,ARG+POLYAMINE-SYN,ARGDEG-PWY,ARGORNPROST-PWY,ARGSYN-PWY,ARGSYNBSUB-PWY,...,THISYN-PWY,THREOCAT-PWY,THRESYN-PWY,TRNA-CHARGING-PWY,TRPSYN-PWY,TYRFUMCAT-PWY,UBISYN-PWY,UDPNAGSYN-PWY,VALDEG-PWY,VALSYN-PWY
116783-plate305,0.006894,0.000000,0.000021,0.007520,0.008957,0.002952,0.000000,0.000345,0.005476,0.005329,...,0.005385,0.000000,0.008002,0.007885,0.005062,0.0,0.000353,0.005087,0.0,0.008914
116784-plate305,0.006424,0.000000,0.000000,0.005238,0.008203,0.002061,0.000000,0.000111,0.004180,0.003247,...,0.005658,0.000000,0.007842,0.007343,0.005919,0.0,0.000061,0.003929,0.0,0.008255
116785-plate305,0.006937,0.000000,0.000000,0.006335,0.008470,0.001367,0.000000,0.000147,0.005108,0.004487,...,0.006907,0.000000,0.008267,0.007242,0.006189,0.0,0.000157,0.002773,0.0,0.008233
116786-plate305,0.006875,0.000000,0.000000,0.006683,0.008730,0.001332,0.000000,0.000121,0.004380,0.003672,...,0.006604,0.000000,0.008148,0.007263,0.006077,0.0,0.000005,0.002675,0.0,0.008247
116787-plate305,0.006765,0.000005,0.000075,0.004806,0.008774,0.002166,0.000008,0.000349,0.005773,0.005783,...,0.004919,0.000014,0.008432,0.008030,0.005553,0.0,0.000330,0.005199,0.0,0.008833


In [227]:
# get filtered meta for float cols only
df_meta_filt = df_meta[float_cols]

# merge OTU and df_meta
df_complete = pd.concat([df_otu, df_meta_filt, df_fxn], axis=1)
df_complete.head()

,k__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanobrevibacter,k__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanosphaera,k__Archaea;p__Euryarchaeota;c__Thermoplasmata;o__E2;f__[Methanomassiliicoccaceae];g__,k__Archaea;p__Euryarchaeota;c__Thermoplasmata;o__E2;f__[Methanomassiliicoccaceae];g__Methanomassiliicoccus,k__Archaea;p__Euryarchaeota;c__Thermoplasmata;o__E2;f__[Methanomassiliicoccaceae];g__vadinCA11,k__Bacteria;__;__;__;__;__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;__;__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Varibaculum,...,THISYN-PWY,THREOCAT-PWY,THRESYN-PWY,TRNA-CHARGING-PWY,TRPSYN-PWY,TYRFUMCAT-PWY,UBISYN-PWY,UDPNAGSYN-PWY,VALDEG-PWY,VALSYN-PWY
116783-plate305,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.005385,0.000000,0.008002,0.007885,0.005062,0.0,0.000353,0.005087,0.0,0.008914
116784-plate305,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.005658,0.000000,0.007842,0.007343,0.005919,0.0,0.000061,0.003929,0.0,0.008255
116785-plate305,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.006907,0.000000,0.008267,0.007242,0.006189,0.0,0.000157,0.002773,0.0,0.008233
116786-plate305,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.006604,0.000000,0.008148,0.007263,0.006077,0.0,0.000005,0.002675,0.0,0.008247
116787-plate305,0.0,0.0,0.0,0.0,0.0,0.000523,0.0,0.0,0.0,0.0,...,0.004919,0.000014,0.008432,0.008030,0.005553,0.0,0.000330,0.005199,0.0,0.008833


In [230]:
# aggregate disease vs healthy analysis first
diag_healthy = ['healthy'] #['unaffected','healthy']
diag_disease = ['RA', 'psa', 'pso', 'ss', 'cd', 'sle']

# 
samp_healthy, samp_disease = [], []
for d in diag_healthy:
    samp_healthy = samp_healthy + diag_to_samp[d]
for d in diag_disease:
    samp_disease = samp_disease + diag_to_samp[d]
    
df_complete_healthy = df_complete.loc[samp_healthy]
df_complete_disease = df_complete.loc[samp_disease]

# save with index_label to avoid CUTIE errors
df_complete_healthy.to_csv(path + 'inputs/df_complete_healthy.tsv', sep='\t', index_label='Samples')
df_complete_disease.to_csv(path + 'inputs/df_complete_disease.tsv', sep='\t', index_label='Samples')

df_complete_healthy


#df_meta['Diagnosis'].value_counts()
#df_disease = df_complete[df_complete['Diagnosis']]
# samp_disease


,k__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanobrevibacter,k__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanosphaera,k__Archaea;p__Euryarchaeota;c__Thermoplasmata;o__E2;f__[Methanomassiliicoccaceae];g__,k__Archaea;p__Euryarchaeota;c__Thermoplasmata;o__E2;f__[Methanomassiliicoccaceae];g__Methanomassiliicoccus,k__Archaea;p__Euryarchaeota;c__Thermoplasmata;o__E2;f__[Methanomassiliicoccaceae];g__vadinCA11,k__Bacteria;__;__;__;__;__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;__;__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Varibaculum,...,THISYN-PWY,THREOCAT-PWY,THRESYN-PWY,TRNA-CHARGING-PWY,TRPSYN-PWY,TYRFUMCAT-PWY,UBISYN-PWY,UDPNAGSYN-PWY,VALDEG-PWY,VALSYN-PWY
MicraHC-1017-B-plate308,0.000000,0.0,0.0,0.0,0.0,0.000552,0.0,0.0,0.000000,0.0,...,0.005016,0.000000,0.008206,0.008231,0.006046,0.000019,0.000672,0.006311,0.0,0.009331
MicraHC-1018-B-plate308,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000803,0.0,...,0.002519,0.000000,0.008178,0.008630,0.005925,0.000000,0.000022,0.005854,0.0,0.010004
MicraHC-1019-B-plate308,0.000000,0.0,0.0,0.0,0.0,0.002330,0.0,0.0,0.000000,0.0,...,0.005564,0.000000,0.007956,0.007582,0.005911,0.000032,0.000276,0.004020,0.0,0.008467
MicraHC-1020-B-plate308,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.002929,0.000013,0.008490,0.008588,0.006640,0.000000,0.000031,0.005973,0.0,0.009986
MicraHC-1021-B-plate308,0.035777,0.0,0.0,0.0,0.0,0.000071,0.0,0.0,0.000000,0.0,...,0.004980,0.000000,0.008410,0.007849,0.004906,0.000035,0.000305,0.005358,0.0,0.009081
SCFA08-B-plate308,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000676,0.0,...,0.004418,0.000373,0.007345,0.007112,0.005346,0.000021,0.000372,0.003905,0.0,0.008251


In [271]:
# plotting differential picrust metacyc pathways
pvals = []
tvals = []
KO = KO_names#[0:1000]

# get OG p values
for k in KO:
    x = df_complete_healthy[k].values
    y = df_complete_disease[k].values
    t, p = scipy.stats.ttest_ind(x,y)
    # remove nans or else, but this might not be the best way
    # https://github.com/statsmodels/statsmodels/issues/2899
    if np.isnan(p):
        pvals.append(1)
    else:
        pvals.append(p)
        tvals.append(t)


# https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.fdrcorrection.html
qvals = statsmodels.stats.multitest.fdrcorrection(np.array(pvals), alpha=0.05, method='indep', is_sorted=False)[1]        
# one day the scipy library will have it (1.11, the next release apparently as of 8/29/2023)
# scipy.stats.false_discovery_control(ps)

df_complete_nonfilt = pd.concat([df_otu, df_meta, df_fxn], axis=1)

try:
    os.mkdir(path + 'outputs/metacycp/')
except:
    pass

try:
    os.mkdir(path + 'outputs/metacycq/')
except:
    pass


# convert to q values
sig_list = []
for i in range(len(KO)):
    p, q = pvals[i], qvals[i]
    k = KO[i]      
    if p < 0.05:        
        sig_list.append(k)
        print(k)
        print(tvals[i], pvals[i])
        sns.set(style="darkgrid")
        plt.subplots(figsize=(6, 4))
        plt.tight_layout()
        ax = sns.boxplot(data=df_complete_nonfilt, x='Diagnosis', y=k)

        # save the plot as PNG file
        plt.savefig(path + 'outputs/metacycp/' + k + '.png',  bbox_inches="tight")
        if q < 0.05:
            plt.savefig(path + 'outputs/metacycq/' + k + '.png',  bbox_inches="tight")
            
        plt.close()
                        
            
print(len(sig_list))



PWY-5419
4.9008927758113625 2.87461771368364e-06
PWY-5420
0.11044462965320417 2.87461771368364e-06
PWY-5507
-0.512801055198138 0.03054921293296315
PWY-6263
-0.35608874253805584 0.006012349996616373
PWY-7371
0.3150097986036645 0.014727287488939521
PWY-7374
0.6522389402536547 0.0008864989382276323
6


In [279]:
# for mapping to cutie indices; both are 0 indexed
# this is for ra
variables = list(df_complete.columns.values)
taxa_interest = ['Prevotella']
for t in variables:
    for i in taxa_interest:
        if i in t:
            print(t)
            print(variables.index(t))


k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Prevotellaceae;g__
36
k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Prevotellaceae;g__Prevotella
37
k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__[Paraprevotellaceae];g__[Prevotella]
55


In [289]:
# jobs01 case by case
df = pd.read_csv(path + 'outputs/jobs00/data_processing/summary_df_resample_1.txt', sep='\t')
df = df[df['pvalues'] < 0.05]
df = df[df['var2'] == 'CCPtiter']
df['var1'].values


array([], dtype=object)